In [62]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time
import statsmodels.formula.api as sm

In [20]:
df = pd.read_csv('hrv_df.csv')

In [26]:
df.head(10)

,Unnamed: 0,user_id,start,end,date_md,score,time_in_bed,latency,disturbances,wake_duration,light_sleep_duration,slow_wave_sleep_duration,rem_sleep_duration,cycles_count,debt_post,is_nap,recovery_score,resting_heart_rate,hrv_rmssd,end_epoch,race_period,buildup_days
0,0,828,9/19/2015 1:44,9/19/2015 11:30,9/19,0,0,0,NaN,0,0,0,0,NaN,NaN,f,NaN,NaN,NaN,1442662200000,1,6
1,1,828,9/20/2015 3:38,9/20/2015 11:44,9/20,70,29156349,1560075,18,4260000,18090000,2910000,3900000,6,7668000.00,f,42,43,0.050816,1442749440000,1,5
2,2,828,9/21/2015 2:30,9/21/2015 12:18,9/21,74,35273083,1525941,23,6270000,16020000,4620000,8370000,6,4792906.02,f,50,45,0.058456,1442837880000,1,4
3,3,828,9/22/2015 2:16,9/22/2015 12:37,9/22,93,37294761,1050923,31,4860000,24000000,3870000,4560000,7,1012272.72,f,69,43,0.063264,1442925420000,1,3
4,4,828,9/23/2015 2:44,9/23/2015 12:20,9/23,87,34522859,870915,23,3630000,22560000,2940000,5400000,10,2610197.56,f,76,47,0.081390,1443010800000,1,2
5,5,828,9/24/2015 3:03,9/24/2015 12:34,9/24,89,34284759,1140930,14,3360000,22140000,3480000,5310000,10,2307830.92,f,74,46,0.076798,1443098040000,1,1
6,6,828,9/25/2015 2:05,9/25/2015 12:57,9/25,97,39120000,1980096,19,8340000,22320000,4440000,4020000,5,463846.66,f,58,43,0.015313,1443185820000,1,0
7,7,828,10/10/2015 3:30,10/10/2015 11:40,10/10,79,29400000,2009009,12,3660000,17550000,3360000,4830000,9,4028457.38,f,37,45,0.051232,1444477200000,2,6
8,8,828,NaN,NaN,10/11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1444531600000,2,5
9,9,828,10/12/2015 3:45,10/12/2015 12:50,10/12,81,32658852,630032,26,2760000,18120000,4920000,6810000,8,4968759.99,f,36,42,0.046391,1444654200000,2,4


In [104]:
groups = df.groupby(['user_id', 'race_period'])
new_df = groups.mean().reset_index()
new_df = new_df.drop(['score', 'disturbances', 'buildup_days', 'wake_duration', 
                      'light_sleep_duration', 'slow_wave_sleep_duration', 'rem_sleep_duration', 'end_epoch', 
                      'cycles_count', 'debt_post'], 1)
new_df = new_df[new_df['race_period']!=0].reset_index(drop=True)

In [105]:
huxc = pd.read_csv('huxc_race_times.csv')
huxc = huxc[huxc['user_id']!=2461].reset_index()
print len(huxc)
print len(new_df)

50
50


In [106]:
final_df = pd.merge(new_df, huxc, on=['user_id', 'race_period'], how="left")
final_df = final_df[final_df['user_id']!=2439]
final_df = final_df[final_df['user_id']!=2508].reset_index(drop=True)
final_df = final_df[final_df['race_period']!=4].reset_index(drop=True)
#final_df = final_df[final_df['race_period']!=2].reset_index(drop=True)
#final_df = final_df[final_df['race_period']!=1].reset_index(drop=True)

In [107]:
final_df.to_csv('test_regression.csv')

In [102]:
#result = sm.ols(formula="seconds_unstd~hrv_rmssd", data=final_df).fit()
#result = sm.ols(formula="seconds_unstd~resting_heart_rate", data=final_df).fit()

from pandas.stats.plm import PanelOLS
groups = final_df.groupby(['user_id', 'race_period'])
#reg = PanelOLS(y=groups['seconds_unstd'], x=groups['hrv_rmssd'], time_effects=True)

In [85]:
result.summary()

C:\Users\BUCKBEAK\Anaconda\lib\site-packages\scipy\stats\stats.py:1277: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          seconds_unstd   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.082
Method:                 Least Squares   F-statistic:                   0.01832
Date:                Thu, 07 Jan 2016   Prob (F-statistic):              0.895
Time:                        14:59:43   Log-Likelihood:                -73.189
No. Observations:                  14   AIC:                             150.4
Df Residuals:                      12   BIC:                             151.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept           1614.8344    127.245     12.691      0.000      1337.592  1892.077
resting_heart_rate    -0.3720      2.748     -0.135      0.895        -6.360     5.616
==============================================================================
Omnibus:                        1.005   Durbin-Watson:                   2.294
Prob(Omnibus):                  0.605   Jarque-Bera (JB):                0.696
Skew:                           0.003   Prob(JB):                        0.706
Kurtosis:                       1.908   Cond. No.                         453.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""